In [ ]:
# default_exp skorch

# Testing skorch

> Why py explored?

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X


net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7018       0.6100        0.6777  0.0416
      2        0.6895       0.6600        0.6645  0.0454
      3        0.6637       0.6800        0.6526  0.0390
      4        0.6624       0.7000        0.6418  0.0380


<ipython-input-4-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
/home/danph/anaconda3/envs/dev/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


      5        0.6438       0.7200        0.6291  0.0392
      6        0.6299       0.7200        0.6195  0.0395
      7        0.6256       0.7200        0.6058  0.0362
      8        0.6072       0.7350        0.5945  0.0419
      9        0.6084       0.7250        0.5890  0.0389
     10        0.5937       0.7400        0.5765  0.0381


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7338       0.5000        0.7109  0.0165
      2        0.7303       0.5000        0.7089  0.0298
      3        0.7305       0.5000        0.7072  0.0255
      4        0.7279       0.5000        0.7059  0.0232
      5        0.7216       0.5000        0.7046  0.0250
      6        0.7242       0.5075        0.7030  0.0221
      7        0.7189       0.5075        0.7020  0.0172


<ipython-input-4-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


      8        0.7193       0.5075        0.7011  0.0298
      9        0.7142       0.5075        0.6999  0.0321
     10        0.7094       0.5075        0.6987  0.0230
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7099       0.4776        0.6918  0.0216
      2        0.7055       0.4925        0.6903  0.0223
      3        0.7023       0.5000        0.6887  0.0253
      4        0.7145       0.4925        0.6883  0.0242
      5        0.7012       0.5149        0.6870  0.0170
      6        0.6991       0.5299        0.6858  0.0303
      7        0.6935       0.5299        0.6852  0.0281


<ipython-input-4-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


      8        0.6980       0.5448        0.6844  0.0158
      9        0.6952       0.5373        0.6837  0.0201
     10        0.6971       0.5373        0.6830  0.0246
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7373       0.4851        0.7197  0.0230
      2        0.7243       0.4851        0.7170  0.0254
      3        0.7267       0.4776        0.7150  0.0254
      4        0.7153       0.4851        0.7131  0.0230
      5        0.7086       0.4851        0.7113  0.0229
      6        0.7123       0.4851        0.7094  0.0243
      7        0.7110       0.4776        0.7080  0.0203
      8        0.7048       0.4851        0.7063  0.0170


...

     10        0.6857       0.5075        0.6847  0.0200
     11        0.6767       0.5075        0.6830  0.0253
     12        0.6819       0.5224        0.6815  0.0153
     13        0.6861       0.5299        0.6803  0.0268
     14        0.6801       0.5821        0.6778  0.0246
     15        0.6759       0.5821        0.6757  0.0167
     16        0.6831       0.5821        0.6740  0.0182
     17        0.6727       0.5821        0.6725  0.0154
     18        0.6749       0.5896        0.6704  0.0188
     19        0.6718       0.5896        0.6687  0.0239
     20        0.6724       0.5970        0.6668  0.0232
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7003       0.4851        0.6998  0.0182
      2        0.6949       0.5075        0.6977  0.0192
      3        0.6894       0.5224        0.6965  0.0225
      4        0.6924       0.5373        0.6952  0.0241
      5        0.6788       0.5

<ipython-input-4-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


      9        0.6837       0.5597        0.6880  0.0170
     10        0.6795       0.5672        0.6864  0.0226
     11        0.6764       0.5896        0.6846  0.0190
     12        0.6784       0.5896        0.6837  0.0232
     13        0.6736       0.5896        0.6825  0.0210
     14        0.6673       0.5896        0.6812  0.0187
     15        0.6622       0.6119        0.6796  0.0226
     16        0.6638       0.6343        0.6777  0.0152
     17        0.6644       0.6269        0.6769  0.0158
     18        0.6635       0.6493        0.6751  0.0177
     19        0.6594       0.6642        0.6737  0.0152
     20        0.6532       0.6642        0.6717  0.0227
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7076       0.5000        0.7047  0.0150
      2        0.6998       0.5000        0.7036  0.0209
      3        0.6988       0.5000        0.7025  0.0212
      4        0.6962       0.5

<ipython-input-4-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


     10        0.6801       0.5075        0.6952  0.0242
     11        0.6804       0.5075        0.6946  0.0165
     12        0.6761       0.5075        0.6940  0.0215
     13        0.6779       0.5149        0.6932  0.0151
     14        0.6695       0.5448        0.6925  0.0208
     15        0.6747       0.5672        0.6915  0.0228
     16        0.6692       0.5522        0.6908  0.0216
     17        0.6760       0.5448        0.6897  0.0222
     18        0.6680       0.5448        0.6888  0.0150
     19        0.6689       0.5672        0.6881  0.0221
     20        0.6641       0.5672        0.6874  0.0145
0.6680392967818117 {'lr': 0.02, 'max_epochs': 10, 'module__num_units': 20}
